# Dating Textual Documents

# The prediction we are hoping to make
Given a short text in English, we wish to perform a classification in predicting the year (or the decade in which) it was written. The writing style of a written work from, say the 1800s, is certainly very different from that of a modern text, and we hope to build a model that can recognize this difference in a consistent way. The related, but different, problem of "author identification" has been tackled before (see e.g. [1] and [2]). 

[1] https://web.stanford.edu/class/cs224n/reports/2760185.pdf <br/>
[2] https://brage.bibsys.no/xmlui/bitstream/handle/11250/2353615/12344_FULLTEXT.pdf?sequence=1

# Data sources
- http://www.gutenberg.org/
- https://books.google.com/ngrams
- https://googlebooks.byu.edu/x.asp (more advanced word tool)
- Google Books (for retrieving publishing dates)
- https://www.goodreads.com/shelf/show/project-gutenberg (a list with 500+ books, the text of which will constitute our dataset)

# Cleaning the data 

<b> Scraping gutenberg.org and goodreads.com to retrieve the relevant texts and their publishing year </b>

From gutenberg.org we want to retrieve as many books as possible in .txt format. The website has about 60 000 books in its catalog, not all of which are of interst to us, e.g. those that are in a different language than English. We think that limiting  our dataset to fiction books with English as its original language makes our dataset more uniform and will prove beneficial for analysis later on. Fortunately, goodreads.com has a catalog which lists 500+ books that more or less satisfy these criteria. We used the data from goodreads.com to obtain the dataset of relevant books (and their publishing year), which we then used to scrape gutenberg.org to retrieve the actual texts. 

<b> Pre-processing a text </b>

Given a written work that can be accessed from gutenberg.org, we wish to (eventually) obtain a dataset which captures word and punctuation frequency, distribution of sentence lengths, and possibly other features. NLTK makes the process of retrieving word frequencies relatively simple. Using the "FreqDist" class, we can produce a list with all words and the number of times they appear in the form of a list of tuples: [(<word>, <word count>), ...]. 

Concerns questions and ideas about how we should clean the data:

1) Do we want all genres of written documents, or just e.g. fictional writing?

2) Stemmatizing/lemmatizing words? 

3) Should we distinguish between lower and uppercase words, or not? Initally we thought it would be a good this to make the set of words uniform in terms of case: consider the case where a word is the first of a sentence and thus have the first of its letter capitalized versus when it does not and have all letters lowercase. In terms of frequency, we'd like to count them as the same word. On the other hand, when it comes to names, we do want to keep the capitalization. It turns out that Google n-grams is case-sensitive. So we definitely want our dataset to be so as well. It really depends on the importance the frequency metric will have for our model. Perhaps we only care about the fact that a word <i> occurs </i> in our dataset, and not its frequency.

4) do we analyze the entirety of the written work, or smaller paragraphs of it? If the paragraphs are too short, this potentially diminishes the informativeness of word frequency; if a paragraph consists of 500 words and 2/3 of these are generic words such as "and", "if" and "but", then perhaps too little room is left for epoch-characteristic words. 

5) Google n-grams "charts the frequencies of any set of comma-delimited search strings using a yearly count of n-grams found in sources printed between 1500 and 2008 in Google's text corpora". However, the frequency for any given year should depend on <i> the number of works that were published in that given year </i>. It most likely is the case that fewer works from the 1500s are incorporated in Google's text corpora than those of the 2000's. So the question is whether Google n-grams really capture the <i> relative commonness </i> of a word between time periods? UPDATE: upon closer inspection, Google n-grams actually measures the <i> relative </i> frequency of a given word for the set of works within given time period.
   

# Data visualization

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('goodreads_catalog.csv')
df.columns = ['title','author','date']

In [6]:
df2 = pd.read_csv('gutenberg_catalog.csv', sep='\t')
df2.columns = ['title','book_id']

In [7]:
combined = pd.merge(df, df2, on='title')

In [8]:
combined.head(5)

,title,author,date,book_id
0,The Mysterious Affair at Styles,Agatha Christie,1920,863.0
1,Pride and Prejudice,Jane Austen,1813,1342.0
2,Pride and Prejudice,Jane Austen,1813,42671.0
3,The Picture of Dorian Gray,Oscar Wilde,1890,174.0
4,The Picture of Dorian Gray,Oscar Wilde,1890,4078.0


In [9]:
combined = combined.drop_duplicates(subset='title')

In [10]:
combined.head(5)

,title,author,date,book_id
0,The Mysterious Affair at Styles,Agatha Christie,1920,863.0
1,Pride and Prejudice,Jane Austen,1813,1342.0
3,The Picture of Dorian Gray,Oscar Wilde,1890,174.0
6,Jane Eyre,Charlotte Brontë,1847,1260.0
7,The Time Machine,H.G. Wells,1895,35.0
